###Murilo Oliveira Rocha
#Atividade Contínua 05
<ul> 
  <li>https://docs.google.com/document/d/1xHGGvZ1pBuCi0zTbo5qb2M2Lg86dYO0NPDNqaxeKvag/edit</li>
</ul>




In [1]:
%%sh
sudo pip install spark
sudo pip install pyspark

  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58762 sha256=0d27c24a8d189e6ee3ecb2f5d80032bd81f15c3914c662ae3156c4b2e42536ac
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f09c4c542ba6d3686cf3ebb88a44d3139c7366a1b39f6f34bb6981e9e92ad88d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pandas as pd

# Importando o spark e o pyspark
import spark,pyspark

# Importando as bibliotecas do pyspark.sql 
from pyspark.sql import *

# Importando as funções sql do spark
# Documentação: https://spark.apache.org/docs/latest/api/sql/index.html
from pyspark.sql import functions as f

# Importando os tipos de dados do spark
# Documentação: https://spark.apache.org/docs/latest/sql-ref-datatypes.html
from pyspark.sql import types as t 

# Biblioteca datetime
from datetime import datetime, date

In [3]:
# Criando uma Sessão do Spark (Spark Session)
spark = SparkSession.builder.master("local").appName("AC5 DataEng").getOrCreate()

In [4]:
# Ler um parquet, passando opções de delimitador e cabeçalho
df= spark.read.format("parquet")\
.option("header","true")\
.option("delimiter",";")\
.load("/content/vendas.parquet")
df.show()

+-------------+----------+-------------+----------+---+---+----------+----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+
|      cod_ean|cod_pessoa|cod_transacao|      data|vlr|qtd|      nome| sms|email|classe|sexo|          cidade|         des_produto|des_familia|   des_secao|des_categoria|des_sub_categoria|__index_level_0__|
+-------------+----------+-------------+----------+---+---+----------+----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+
|7896901200013|      4644|      8064284|2020-10-26| 67|  3|NOME_a4644|true|false|  Ouro|   F|       São Paulo|                null|       null|        null|         null|             null|                0|
|7897001010014|      3578|      8067745|2020-10-26| 54|  7|NOME_o3578|true|false|  Ouro|   M|  Belo Horizonte|                null|       null|        null|         null|  

In [7]:
# Convertendo a coluna data para DateType
df = df.withColumn("data",df.data.cast(t.DateType()))

df.printSchema()

root
 |-- cod_ean: string (nullable = true)
 |-- cod_pessoa: long (nullable = true)
 |-- cod_transacao: long (nullable = true)
 |-- data: date (nullable = true)
 |-- vlr: long (nullable = true)
 |-- qtd: long (nullable = true)
 |-- nome: string (nullable = true)
 |-- sms: boolean (nullable = true)
 |-- email: boolean (nullable = true)
 |-- classe: string (nullable = true)
 |-- sexo: string (nullable = true)
 |-- cidade: string (nullable = true)
 |-- des_produto: string (nullable = true)
 |-- des_familia: string (nullable = true)
 |-- des_secao: string (nullable = true)
 |-- des_categoria: string (nullable = true)
 |-- des_sub_categoria: string (nullable = true)
 |-- __index_level_0__: long (nullable = true)



In [17]:
# Criar as colunas, utilizando as funções spark.sql.functions.year, spark.sql.functions.month , spark.sql.functions.dayofMonth
df = df.withColumn("ano",f.year("data"))
df = df.withColumn("mes",f.month("data"))
df = df.withColumn("dia",f.dayofmonth("data"))
df.show()

+-------------+----------+-------------+----------+---+---+----------+----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+----+---+---+
|      cod_ean|cod_pessoa|cod_transacao|      data|vlr|qtd|      nome| sms|email|classe|sexo|          cidade|         des_produto|des_familia|   des_secao|des_categoria|des_sub_categoria|__index_level_0__| ano|mes|dia|
+-------------+----------+-------------+----------+---+---+----------+----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+----+---+---+
|7896901200013|      4644|      8064284|2020-10-26| 67|  3|NOME_a4644|true|false|  Ouro|   F|       São Paulo|                null|       null|        null|         null|             null|                0|2020| 10| 26|
|7897001010014|      3578|      8067745|2020-10-26| 54|  7|NOME_o3578|true|false|  Ouro|   M|  Belo Horizonte|          

In [18]:
# Na coluna sms, se caso o valor for null , troque por False (use a função spark.sql.functions.when (df.sms.isNull(),False).otherwise(df.sms)) 
df = df.withColumn("sms",f.when(df.sms.isNull(),False).otherwise(df.sms))
df.show()

+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+----+---+---+
|      cod_ean|cod_pessoa|cod_transacao|      data|vlr|qtd|      nome|  sms|email|classe|sexo|          cidade|         des_produto|des_familia|   des_secao|des_categoria|des_sub_categoria|__index_level_0__| ano|mes|dia|
+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+----+---+---+
|7896901200013|      4644|      8064284|2020-10-26| 67|  3|NOME_a4644| true|false|  Ouro|   F|       São Paulo|                null|       null|        null|         null|             null|                0|2020| 10| 26|
|7897001010014|      3578|      8067745|2020-10-26| 54|  7|NOME_o3578| true|false|  Ouro|   M|  Belo Horizonte|     

In [21]:
# Crie uma coluna chamada “total” que deve ser a multiplicação das colunas “vlr” e “qtd”.
df = df.withColumn("total", df.vlr * df.qtd)
df.show()

+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+----+---+---+-----+
|      cod_ean|cod_pessoa|cod_transacao|      data|vlr|qtd|      nome|  sms|email|classe|sexo|          cidade|         des_produto|des_familia|   des_secao|des_categoria|des_sub_categoria|__index_level_0__| ano|mes|dia|total|
+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+----------------+--------------------+-----------+------------+-------------+-----------------+-----------------+----+---+---+-----+
|7896901200013|      4644|      8064284|2020-10-26| 67|  3|NOME_a4644| true|false|  Ouro|   F|       São Paulo|                null|       null|        null|         null|             null|                0|2020| 10| 26|  201|
|7897001010014|      3578|      8067745|2020-10-26| 54|  7|NOME_o3578| true|false|  Ouro|   

In [23]:
# Grave o dataframe particionado por ano, mes e dia no formato “parquet” de nome “vendas_tratadas”.
df.write.partitionBy("ano","mes","dia").parquet("/content/vendas_tratadas")

In [25]:
# Crie uma tabela temporaria chamada “vendas_table” (df.registerTempTable(“vendas_table”)).
df_temp = df.createOrReplaceTempView("vendas_table")
df_temp

In [27]:
# Crie um dataframe spark.sql utilizando a query spark.sql(“select distinct cidade from vendas_table”)
df_cidade = spark.sql("select distinct cidade from vendas_table")
df_cidade.show()

+----------------+
|          cidade|
+----------------+
|          Santos|
|        Curitiba|
|            null|
|        Campinas|
|  Belo Horizonte|
|        Salvador|
|     Campo Largo|
|       São Paulo|
|          Recife|
|          Manaus|
|  Rio de Janeiro|
|    Campo Alegre|
|São Bento do Sul|
|    Porto Alegre|
+----------------+



In [28]:
# Grave esse dataframe no formato “parquet” de nome “cidade”.
df_cidade.write.mode("overwrite").parquet("/content/cidade")

In [63]:
from pyspark.sql.functions import trim
# Crie um novo dataframe chamado “produto” selecionando as colunas: “des_produto”,”des_familia”, “des_secao”, “des_categoria”,”des_sub_categoria”. 
# df_produto = df.select("des_produto","des_familia", "des_secao", "des_categoria", "des_sub_categoria")
df_produto = df.select(f.trim(df['des_produto']).alias('des_produto'),
                f.trim(df['des_secao']).alias('des_secao'),
                f.trim(df['des_categoria']).alias('des_categoria'),
                f.trim(df['des_sub_categoria']).alias('des_sub_categoria')
                )
df_produto.show()

+--------------------+------------+-------------+-----------------+
|         des_produto|   des_secao|des_categoria|des_sub_categoria|
+--------------------+------------+-------------+-----------------+
|                null|        null|         null|             null|
|                null|        null|         null|             null|
|PAO DA CASA FRANC...|PADARIA LOJA|PRODUCAO PAES|                 |
|                null|        null|         null|             null|
| ACEM BOVINO KG PECA|     ACOUGUE| CARNE BOVINA|                 |
|                null|        null|         null|             null|
|                null|        null|         null|             null|
| TOMATE PIZZADORO KG|       F L V|      LEGUMES|                 |
|  CEBOLA NACIONAL KG|       F L V|       CEBOLA|                 |
|  CEBOLA NACIONAL KG|       F L V|       CEBOLA|                 |
|   ALHO GRANEL T6 KG|       F L V|         ALHO|                 |
| TOMATE PIZZADORO KG|       F L V|      LEGUMES

In [65]:
df_produto.distinct()

DataFrame[des_produto: string, des_secao: string, des_categoria: string, des_sub_categoria: string]

In [67]:
# Grave esse dataframe no formato “parquet” de nome produto.
df_produto.write.mode("overwrite").parquet("/content/produto")

In [76]:
# Crie um dataframe chamado “venda_mulheres”, selecionando todas as colunas e filtrando apenas vendas realizadas pelo sexo “F”,
df_mulheres = df.where(df.sexo=='F')
df_mulheres.show()

+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+----------------+--------------------+-----------+------------+-------------------+-----------------+-----------------+----+---+---+-----+
|      cod_ean|cod_pessoa|cod_transacao|      data|vlr|qtd|      nome|  sms|email|classe|sexo|          cidade|         des_produto|des_familia|   des_secao|      des_categoria|des_sub_categoria|__index_level_0__| ano|mes|dia|total|
+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+----------------+--------------------+-----------+------------+-------------------+-----------------+-----------------+----+---+---+-----+
|7896901200013|      4644|      8064284|2020-10-26| 67|  3|NOME_a4644| true|false|  Ouro|   F|       São Paulo|                null|       null|        null|               null|             null|                0|2020| 10| 26|  201|
|         8555|      6505|      8301911|2020-12-18|  6|  1|NOME_i650

In [77]:
# grave esse dataframe no formato json “venda_mulher”. (df.write.format(“json”).save(“venda_mulher”)).
df_mulheres.write.format("json").save("venda_mulher")

In [90]:
# Crie um dataframe chamado “venda_unicas”, selecionando todas as colunas e filtrando apenas vendas com a qtd igual 1 e o vlr menor que 10. 
df_venda_unicas = df.where((df.qtd  == 1) & (df.vlr  < 10))
df_venda_unicas.show()

+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+--------------+--------------------+-----------+------------+-------------------+-----------------+-----------------+----+---+---+-----+
|      cod_ean|cod_pessoa|cod_transacao|      data|vlr|qtd|      nome|  sms|email|classe|sexo|        cidade|         des_produto|des_familia|   des_secao|      des_categoria|des_sub_categoria|__index_level_0__| ano|mes|dia|total|
+-------------+----------+-------------+----------+---+---+----------+-----+-----+------+----+--------------+--------------------+-----------+------------+-------------------+-----------------+-----------------+----+---+---+-----+
|         8555|      6505|      8301911|2020-12-18|  6|  1|NOME_i6505| true|false|  Ouro|   F|Rio de Janeiro|PAO DA CASA FRANC...| PERECIVEIS|PADARIA LOJA|      PRODUCAO PAES|                 |                2|2020| 12| 18|    6|
|         3690|      3618|      8025931|2020-10-17|  3|  1|NOME_e3618| true|

In [92]:
# Grave esse dataframe no formato csv “venda_unica”. (df.write.format(“csv”).save(“venda_unica”)).
df_venda_unicas.write.format("csv").save("venda_unica")